# Pose Operation

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com weikunwu@163.com


Pose中储存了非常多的信息，同时还提供了接口可以让用户方便的对其中的信息进行修改（采样）。
本章节将着重介绍Pose的基本操作方式。

In [25]:
# 首先依然是从PDB中读入Pose
from pyrosetta import init, pose_from_pdb
init()
pose = pose_from_pdb('./data/4R80.clean.pdb')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1224136651 seed_offset=0 real_seed=-1224136651 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1224136651 RG_type=mt19937
core.import_pose.impor

### 1 Pose的创建和复制
前几节中提及过，pose是一个容器，理所当然我们可以创建一个空的容器，里面什么都不放。</br>
很多时候，我们需要创建空的Pose对象，便于保存当前pose的实例化状态，可作为可回溯点或初始状态，方便反复调用。

In [23]:
# 通过创建一个新的Pose
from pyrosetta import Pose
starting_pose = Pose()
starting_pose2 = None

此处通过两种方法，将已有的Pose信息放入新的容器里，一种是通过assign函数复制，一种是通过python赋值符号来赋值。

In [24]:
# 方法1：通过assign复制新的构象
starting_pose.assign(pose)

# 方法2：通过python的直接赋值符号
starting_pose2 = pose

print(starting_pose)
print('\n')
print(starting_pose2)

Total residues: 76
Sequence: PSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGS
Fold tree:
FOLD_TREE  EDGE 1 76 -1 


Total residues: 76
Sequence: PSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGS
Fold tree:
FOLD_TREE  EDGE 1 76 -1 


可见，两种方式“看”起来里面都有了新的pose信息。但真的如此么？

In [8]:
# 尝试调整mypose中的24号氨基酸phi值:
pose.set_phi(24, 170.0)

# 查看对starting_pose以及starting_pose2的影响:
print(f'starting_pose 24 residue phi:{starting_pose.phi(24)}')
print(f'starting_pose2 24 residue phi:{starting_pose2.phi(24)}')

starting_pose 24 residue phi:-91.24043946940483
starting_pose2 24 residue phi:170.0


**结果表明:
starting_pose2是用过python直接赋值的Pose并没有复制，而只是pose的一个"超链接"符，并没有进行"复制"的操作。
而通过assign的复制，原始的pose的任何调整都没有对starting_pose造成任何的影响，可见其独立性。**

### 2. 链与氨基酸的增减操作
先前章节中描述的都是对蛋白质的几何空间做参数的调整，而并不涉及到拓扑结构的改变。在PyRosetta中，对蛋白的结构域进行修改本质上就是对氨基酸的插入和删除。

#### 2.1 链交换

In [64]:
# 链交换, 注意链号会变成AB.
from pyrosetta.rosetta.protocols.simple_moves import SwitchChainOrderMover
exchange_pose = pose_from_pdb('./data/6LZ9_H_L.pdb')
print(exchange_pose.pdb_info())
switch_chains = SwitchChainOrderMover()
switch_chains.chain_order("LH")
switch_chains.apply(exchange_pose)
print(exchange_pose.pdb_info())

core.import_pose.import_pose: {0} File './data/6LZ9_H_L.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 21 94
core.conformation.Conformation: {0} current variant for 21 CYS
core.conformation.Conformation: {0} current variant for 94 CYS
core.conformation.Conformation: {0} current variant for 21 CYD
core.conformation.Conformation: {0} current variant for 94 CYD
core.conformation.Conformation: {0} Found disulfide between residues 141 206
core.conformation.Conformation: {0} current variant for 141 CYS
core.conformation.Conformation: {0} current variant for 206 CYS
core.conformation.Conformation: {0} current variant for 141 CYD
core.conformation.Conformation: {0} current variant for 206 CYD
PDB file name: ./data/6LZ9_H_L.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0081    H 0002  -- 0082  |   0081 residues;    01283 atoms
0082 -- 0082    H 0082A -- 0082A |   0001 residues;    00011 atoms
0083

#### 2.2 链删除

In [22]:
# 链删除
from pyrosetta.rosetta.protocols.simple_moves import DeleteChainMover
delete_chain = DeleteChainMover()
delete_chain.chain_num(1)
delete_chain.apply(pose)
print(pose.pdb_info())

protocols.simple_moves.DeleteChainMover: {0} Removing chain 1 from pose with 1 chains
None


#### 2.3 链的索引
尽管pose的氨基酸编号是忽略多肽链的分隔的，但是pose中的链依然是根据多肽链的物理结构进行编号的，同理也是从1开始编号。
如一个蛋白中有2条链A和B，那么链编号结果即为1和2。其中A对应1号链，B对应2号链，与PDB的链顺序有关（当然A链的顺序如果在后面，那么B链就是1号链）。
Pose类中许多的方法可以很方便对链的增减进行操作, 以下2个举例进行说明:

In [41]:
# 将Pose按照链的数量进行切割
pose = pose_from_pdb('./data/4R80.clean.pdb')
pose_list = pose.split_by_chain()
pose_list

core.import_pose.import_pose: {0} File './data/4R80.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 152


vector1_std_shared_ptr_core_pose_Pose_t[0x7f86eb2a00a0, 0x7f86ed4a40c0]

此处的pose_list中存放了2个数据，说明链已经被切割成2个独立的pose对象了。

通过python的索引，可以获得具体的pose:

In [42]:
# 获取只含有第一个链的pose
chain1_pose = pose.split_by_chain()[1]  # 直接切片索引链号。
chain2_pose = pose.split_by_chain()[2]  # 直接切片索引链号。

# check
print(f'chain1_pose中的氨基酸总数:{chain1_pose.total_residue()}')
print(f'chain2_pose中的氨基酸总数:{chain2_pose.total_residue()}')
print(f'原始pose中的氨基酸总数:{pose.total_residue()}')

chain1_pose中的氨基酸总数:76
chain2_pose中的氨基酸总数:76
原始pose中的氨基酸总数:152


#### 2.4 链的合并或延伸
除了分隔操作，用户还可以通过一些简单的方式把链合并到一个pose中，此处使用append_pose_to_pose函数就可以达到目的。但需要注意，pose中的氨基酸、链的数量变化后，都需要对PDBinfo进行更新。否则PDBinfo的信息与Pose信息不对称。

In [43]:
# 两条链的合并;
# add binder to pose;
from pyrosetta.rosetta.core.pose import append_pose_to_pose
combine_pose = Pose()
combine_pose.assign(chain1_pose)
print(f'原始chain1_pose中的氨基酸总数:{chain1_pose.total_residue()}')
append_pose_to_pose(combine_pose, chain2_pose)
combine_pose.update_residue_neighbors()
combine_pose.update_pose_chains_from_pdb_chains()
combine_pose.conformation().detect_disulfides()

# update pdbinfo; 别忘了更新pdbinfo;
# 更新pdb_info; [别忘了]
from pyrosetta.rosetta.core.pose import renumber_pdbinfo_based_on_conf_chains
renumber_pdbinfo_based_on_conf_chains(combine_pose)

print(f'append之后的chain1_pose中的氨基酸总数:{combine_pose.total_residue()}')
combine_pose.sequence()

# 检查PDBinfo是否正确: Returns true if PDBInfo is obsolete and needs updating
print(f'PDBinfo是否需要被更新:{combine_pose.pdb_info().obsolete()}')

原始chain1_pose中的氨基酸总数:76
append之后的chain1_pose中的氨基酸总数:152
PDBinfo是否需要被更新:False


In [45]:
# 链的延伸; 合并成一条链的意思;
new_chain = False
combine_pose = Pose()
combine_pose.assign(chain1_pose)
append_pose_to_pose(combine_pose, chain2_pose, new_chain)

# update pdbinfo; 别忘了更新pdbinfo;
# 更新pdb_info; [别忘了]
from pyrosetta.rosetta.core.pose import renumber_pdbinfo_based_on_conf_chains
renumber_pdbinfo_based_on_conf_chains(combine_pose)

print(f'append之后的chain1_pose中的氨基酸总数:{combine_pose.total_residue()}')
combine_pose.sequence()

# 检查PDBinfo是否正确: Returns true if PDBInfo is obsolete and needs updating
print(f'PDBinfo是否需要被更新:{combine_pose.pdb_info().obsolete()}')

append之后的chain1_pose中的氨基酸总数:152
PDBinfo是否需要被更新:False


#### 2.5 氨基酸的删减操作

除了对链的合并之外，我们还可以对链中的氨基酸进行添加、删除的操作！具体的过程是用户需要创建一个独立的氨基酸(residue object)，并将这个氨基酸加载到现有的构像中。
加载的方式可以是前置后后置，根据使用的函数不同而定。

In [15]:
# 在链的前端添加新的氨基酸或删除氨基酸
from pyrosetta.rosetta.core.conformation import ResidueFactory
from pyrosetta.rosetta.core.chemical import ChemicalManager

print(f'原始氨基酸总数:{pose.total_residue()}')
print(f'原始氨基酸序列:{pose.sequence()}\n')

# 向前添加氨基酸
chm = ChemicalManager.get_instance()
rts = chm.residue_type_set("fa_standard")
new_rsd = ResidueFactory.create_residue(rts.name_map('ALA')) # 创建一个residue object
pose.prepend_polymer_residue_before_seqpos(new_rsd, 1, True)  # 在第一个氨基酸前添加一个ALA

print(f'向前添加之后氨基酸总数:{pose.total_residue()}')
print(f'向前添加之后氨基酸序列:{pose.sequence()}\n')

原始氨基酸总数:152
原始氨基酸序列:PSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSPSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGS

core.conformation.Residue: {0} [ WARNING ] Residue connection id changed when creating a new residue at seqpos 1
core.conformation.Residue: {0} [ WARNING ] ResConnID info stored on the connected residue (residue 2) is now out of date!
core.conformation.Residue: {0} [ WARNING ] Connection atom name (in src):  C
向前添加之后氨基酸总数:153
向前添加之后氨基酸序列:APSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSPSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGS



In [16]:
# 向后添加氨基酸
last_residue = pose.total_residue()
pose.append_polymer_residue_after_seqpos(new_rsd, last_residue, True)  # 在第一个氨基酸前添加一个ALA

print(f'向后添加之后氨基酸总数:{pose.total_residue()}')
print(f'向后添加之后氨基酸序列:{pose.sequence()}\n')

向后添加之后氨基酸总数:154
向后添加之后氨基酸序列:APSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSPSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSA



In [17]:
# 删除氨基酸
pose.delete_polymer_residue(1)  # 删除第一个氨基酸

print(f'删除之后氨基酸总数:{pose.total_residue()}')
print(f'删除之后氨基酸序列:{pose.sequence()}\n')

删除之后氨基酸总数:153
删除之后氨基酸序列:PSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSPSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSA



In [18]:
# 还可以范围性的删除氨基酸
pose.delete_residue_range_slow(1,5) # 删除第一个至第五个氨基酸

print(f'删除之后氨基酸总数:{pose.total_residue()}')
print(f'删除之后氨基酸序列:{pose.sequence()}\n')

删除之后氨基酸总数:148
删除之后氨基酸序列:EKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSPSEEEEKRRAKQVAKEKILEQNPSSKVQVRRVQKQGNTIRVELEITENGKKTNITVEVEKQGNTFTVKRITETVGSA



**对拓扑结构操作完后，千万记得PBDinfo更新!!**

In [19]:
# 更新pdb_info; [别忘了!]
from pyrosetta.rosetta.core.pose import renumber_pdbinfo_based_on_conf_chains

renumber_pdbinfo_based_on_conf_chains(pose)  # 更新PDBinfo.

# 检查PDBinfo是否正确: Returns true if PDBInfo is obsolete and needs updating
print(f'PDBinfo是否需要被更新:{pose.pdb_info().obsolete()}')

PDBinfo是否需要被更新:False


#### 2.6 删除Region


In [58]:
pose = pose_from_pdb('./data/4R80.clean.pdb')
print('删除前的PDB_info:')
print(pose.pdb_info())
from pyrosetta.rosetta.protocols.grafting import delete_region
delete_region_mover = delete_region(pose, 1, 50)  # 此处是pose number;

# 打印查看信息, 一整个区域都被删除【比一个个氨基酸删除快得多】:
print('删除后的PDB_info:')
print(pose.pdb_info())

core.import_pose.import_pose: {0} File './data/4R80.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 152
删除前的PDB_info:
PDB file name: ./data/4R80.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0076    A 0001  -- 0076  |   0076 residues;    01251 atoms
0077 -- 0152    B 0001  -- 0076  |   0076 residues;    01251 atoms
                           TOTAL |   0152 residues;    02502 atoms

protocols.grafting.util: {0} Deleting 50 residues from 1 to 50
删除后的PDB_info:
PDB file name: ./data/4R80.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0026    A 0051  -- 0076  |   0026 residues;    00412 atoms
0027 -- 0102    B 0001  -- 0076  |   0076 residues;    01251 atoms
                           TOTAL |   0102 residues; 

#### 2.7 氨基酸类型的调整(突变)
除了具体的化学键数据的调整，在PyRosetta中进行氨基酸的类型调整也是很方便的

In [20]:
# 调整氨基酸的类型
from pyrosetta.toolbox import mutate_residue
print(f'原始氨基酸类型:{pose.residue(1).name()}')
print('突变氨基酸中...')
mutate_residue(pose, 1, 'A', 9.0)  # 1 代表氨基酸突变的pose编号，9.0代表对氨基酸附近9埃范围内的氨基酸进行侧链优化，适应新的突变。
print(f'突变后氨基酸类型:{pose.residue(1).name()}')

原始氨基酸类型:GLU
突变氨基酸中...
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.i

### 3.自定义信息
Pose中含有让用户自定义写入任何信息的功能，比如在程序设计过程中，中间生成的临时数值或字符都可以写入到PoseExtraScore中，这些信息会随着Pose一并输出到PDB或则Silent文件中，在后续的分析和处理的过程中非常方便。

In [21]:
# 给pose加入额外的信息: 比如filter计算的值就可以储存.
from pyrosetta.rosetta.core.pose import setPoseExtraScore, getPoseExtraScore

setPoseExtraScore(pose, "distance", 1.0)
setPoseExtraScore(pose, "angle", 120.5)

# 提取信息
print(getPoseExtraScore(pose, 'distance'))
print(getPoseExtraScore(pose, 'angle'))  # 目前有bug，但是信息已经储存在pose中了

1.0
120.5
